In [2]:
import pandas as pd

In [40]:
ratio_params = {
    'D': 0.01,
    'C': 0.1,
    'B': 0.2,
    'A': 0.3
}

def rate_pos(ratio):
    curr_grade = 'E'
    for grade in ratio_params.keys():
        if ratio < ratio_params[grade]:
            break
        curr_grade = grade
    return curr_grade

In [64]:
# from FanGraphs, get DWAR usingTY data
def read_pos(year):
    df = pd.read_csv(f'DWAR {year - 1}-{year}.csv', index_col=False).sort_values('Inn', ascending=False)#.drop_duplicates(subset='playerid', keep='last')
    sums = df.groupby('playerid').sum()[['Inn']].reset_index().rename({'Inn': 'TotalInn'}, axis=1)
    df = df.merge(sums, how='left', on='playerid')
    df['InnRatio'] = df['Inn'] / df['TotalInn']
    df['Rating'] = df.apply(lambda x: rate_pos(x['InnRatio']), axis=1)
    df = df[['playerid', 'Pos', 'Rating']]    
        
    # join with player ids
    player_map = pd.read_csv('../PLAYERIDMAP 2022.csv')[['IDFANGRAPHS', 'PLAYERNAME', 'MLBID']]#.set_index('IDFANGRAPHS')
        
    df['playerid'] = df['playerid'].astype(str)
    
    merge = df.merge(player_map, how='left', left_on='playerid', right_on='IDFANGRAPHS')
    
    data = merge[['MLBID', 'Pos', 'Rating']]
    
    pos_df = []
    
    for mlbid in data['MLBID'].unique():
        player = list(data.loc[data['MLBID'] == mlbid]['Pos'])
        c = 'F' if 'C' not in player else list(data.loc[(data['MLBID'] == mlbid) & (data['Pos'] == 'C'), 'Rating'])[0]
        first = 'F' if '1B' not in player else list(data.loc[(data['MLBID'] == mlbid) & (data['Pos'] == '1B'), 'Rating'])[0]
        second = 'F' if '2B' not in player else list(data.loc[(data['MLBID'] == mlbid) & (data['Pos'] == '2B'), 'Rating'])[0]
        third = 'F' if '3B' not in player else list(data.loc[(data['MLBID'] == mlbid) & (data['Pos'] == '3B'), 'Rating'])[0]
        short = 'F' if 'SS' not in player else list(data.loc[(data['MLBID'] == mlbid) & (data['Pos'] == 'SS'), 'Rating'])[0]
        left = 'F' if 'LF' not in player else list(data.loc[(data['MLBID'] == mlbid) & (data['Pos'] == 'LF'), 'Rating'])[0]
        center = 'F' if 'CF' not in player else list(data.loc[(data['MLBID'] == mlbid) & (data['Pos'] == 'CF'), 'Rating'])[0]
        right = 'F' if 'RF' not in player else list(data.loc[(data['MLBID'] == mlbid) & (data['Pos'] == 'RF'), 'Rating'])[0]
        pos_df.append({
            'MLBID': mlbid,
            'C': c,
            '1B': first,
            '2B': second,
            '3B': third,
            'SS': short,
            'LF': left,
            'CF': center,
            'RF': right
        })
    
    return pd.DataFrame.from_records(pos_df)

pos_data = read_dwar(2022)
pos_data.head()

,MLBID,C,1B,2B,3B,SS,LF,CF,RF
0,621566.0,F,A,F,F,F,F,F,F
1,621020.0,F,F,F,F,A,F,F,F
2,663586.0,F,F,F,A,F,F,F,F
3,607208.0,F,F,F,F,A,F,F,F
4,596019.0,F,F,F,F,A,F,F,F


In [69]:
pos_data.to_csv('2022_Positions.csv', index=False)